In [90]:
# 1. 웹에서 html 문서를 가져와 beautifulsoup 으로 파싱
from bs4 import BeautifulSoup
import urllib.request  # 웹상의 url 을 파이썬이 인식할 수 있도록 해주는 모듈
import pandas as pd

sub_total=pd.DataFrame()
res_list=[]
for i in range(1,1000):
    list_url = ('https://m.jobkorea.co.kr/Start/spec/view?C_Idx='+ str(i) +'&Tab=3&VPage=1')
    url = urllib.request.Request(list_url)

    result = urllib.request.urlopen(url).read().decode("utf-8")
    soup = BeautifulSoup( result, "html.parser")
    # 2. 시청자 게시판의 날짜와 본문 내용을 가져옵니다.
    result1 = soup.find('h2', class_='co')  # 회사이름
    result2 = soup.find_all('span', class_='score') # 점수들
    result3 = soup.find_all('span', class_='cate') # 지원분야
    result4 = soup.find_all('span', class_='txYear') # 시기
    middle5 = soup.find('ul', class_='list')
    if middle5 is not None:
        result5 = middle5.find_all('span', class_='tx') # 대학, 계열, 전공

    if result1 and result5 is not None:
        # 3. 리스트에 담습니다.
        params1 = []
        params1.append(result1.get_text("  ", strip=True))
        params1=[params1]   # 회사명
        
        params3= []
        sub_li3=[]  # 지원분야
        
        params4= []
        sub_li4=[]  # 시기 년만
        se_li4=[]
        
        params5= []
        sub_li5=[]  # 대학 계열 전공 분류
        se_li5=[]


        params2 = []
        sub_li2=[] # 회사별 데이터

        list3=[]  # 회사명 + 회사별 데이터
        
        
        for j in result2:
            params2.append(j.get_text("  ", strip=True)) # Score 1페이지에 있는 것들 나옴
        for j in range(0,len(params2),10):
            sub_li2.append(params2[j:j+10])
        sub_li2.pop(-1) # 회사별 데이터
        
        for j in result3:
            params3.append(j.get_text("  ", strip=True)) # Score 1페이지에 있는 것들 나옴
        for j in range(len(params3)):
            sub_li3.append([params3[j]]) # 지원분야
        
        for j in result4:
            params4.append(j.get_text("  ", strip=True)) # Score 1페이지에 있는 것들 나옴
        for i in params4:
            se_li4.append(i.split(' '))
        sub_li4 = [[item[0]] for item in se_li4] # 시기

        for j in result5:
            params5.append(j.get_text("  ", strip=True)) # Score 1페이지에 있는 것들 나옴
        # 분류
        for i in params5:
            se_li5.append(i.split(' '))
        
        sub_li5 = [[item[0]] for item in se_li5] # 대학 
        
        sub_li6 = [[item[2]] for item in se_li5] # 계열
        for sublist in sub_li6:
            if '스펙지수' in sublist:
                sublist[0] = '-'
       
        sub_li7 = [[item[3]] for item in se_li5]  # 전공

        list3 = [item1 + item2 + item3 + item4 + item5 + item6 + item7 for item1, item2, item3, item4, item5, item6, item7 in zip(params1 * len(sub_li2),sub_li4,sub_li3,sub_li5,sub_li6,sub_li7, sub_li2)]
        res_list.append(list3)

IncompleteRead: IncompleteRead(37960 bytes read, 212056 more expected)

In [91]:
data_2d = [inner_list for outer_list in res_list for inner_list in outer_list]
df=pd.DataFrame(data_2d,columns=['회사명','시기','지원분야','대학','계열','전공','학점','토익','토익스피킹','오픽','외국어','자격증','해외경험','인턴','수상내역','봉사'])

In [92]:
df.to_csv('./잡코리아취업데이터종합추추추가.csv', encoding = 'utf-8-sig')
# , encoding='EUC-KR'